### LLM powered chatbot


In [105]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")


In [106]:
from langchain_groq import ChatGroq
model= ChatGroq(model="gemma2-9b-it", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F4D310F9A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F4D326BB80>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [107]:
from langchain_core.messages import HumanMessage
model.invoke(
    [
        HumanMessage(content = "Hi, My name is Aijul and I am a DATA Scientist")
    ]
)

AIMessage(content="Hello Aijul, it's nice to meet you!\n\nThat's awesome, Data Science is a fascinating field.  \n\nWhat kind of data science work are you involved in? Are you working on any interesting projects right now?  \n\nI'm always eager to learn more about what people are doing in this area.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 23, 'total_tokens': 94, 'completion_time': 0.129090909, 'prompt_time': 0.002053618, 'queue_time': 0.24442804599999998, 'total_time': 0.131144527}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--56b6e3b0-0532-4348-875e-c06f87e54d18-0', usage_metadata={'input_tokens': 23, 'output_tokens': 71, 'total_tokens': 94})

In [108]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, My name is Aijul and I am a DATA Scientist"),
        AIMessage(content="Hello Aijul, it's nice to meet you! \n\nBeing a Data Scientist is an exciting field. What kind of projects are you currently working on?\n\nI'm eager to learn more about your work and maybe even discuss some data science topics with you.  \n\n"),
        HumanMessage(content="hey, whats my name and what i do?")
    ]
)

AIMessage(content="You are Aijul, and you are a Data Scientist!  \n\nI remember that from our earlier conversation. 😊  \n\nIs there anything else you'd like to talk about?  Perhaps you could tell me about a project you're working on?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 101, 'total_tokens': 157, 'completion_time': 0.101818182, 'prompt_time': 0.004956476, 'queue_time': 0.243373897, 'total_time': 0.106774658}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--4ae22370-9db2-4e3e-adb7-9b7d51dc3ea2-0', usage_metadata={'input_tokens': 101, 'output_tokens': 56, 'total_tokens': 157})

In [109]:
## message history
!pip install langchain_community

In [110]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


store = {}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]= ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)


In [111]:
config = {"configurable": {"session_id":"chat1"}}

In [112]:
response= with_message_history.invoke(
    [
        HumanMessage(content="Hi, My name is Aijul and I am a DATA Scientist")],
        config=config
)

In [113]:
response.content

"Hi Aijul, it's nice to meet you! \n\nIt's great to know you're a data scientist. That's a fascinating field.  \n\nWhat kind of data science work are you involved in? Perhaps you could tell me about a project you're particularly proud of, or a challenge you're currently facing. I'm always eager to learn more about what people are doing in this area.\n"

In [114]:
with_message_history.invoke(
    [
        HumanMessage(content="what is my name")],
        config=config
)

AIMessage(content='Your name is Aijul.  \n\nYou told me at the beginning of our conversation! 😊  \n\n\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 127, 'total_tokens': 152, 'completion_time': 0.045454545, 'prompt_time': 0.008212113, 'queue_time': 0.29270159599999995, 'total_time': 0.053666658}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--205d134c-beb9-46bb-bb55-c615fd5aea2e-0', usage_metadata={'input_tokens': 127, 'output_tokens': 25, 'total_tokens': 152})

In [115]:
config1 = {}

In [116]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a helpful assistant. Answer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [117]:
chain.invoke({"messages": [HumanMessage(content="Hi my name is Aijul")]})

AIMessage(content="Hello Aijul, it's nice to meet you!\n\nI'm happy to help with any questions you have. Just ask away! 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 32, 'total_tokens': 67, 'completion_time': 0.063636364, 'prompt_time': 0.002255918, 'queue_time': 0.244969306, 'total_time': 0.065892282}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--87e740de-ee45-40dd-9e27-2090a398fda3-0', usage_metadata={'input_tokens': 32, 'output_tokens': 35, 'total_tokens': 67})

In [118]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [119]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
#add complexity
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a helpful assistant. Answer all the question to the nest of your ability in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [120]:
response=chain.invoke({"messages":[HumanMessage(content="Hi my name is Aijul")], "language":"Hindi"})

In [121]:
response.content

'नमस्ते Aijul!  \n\nआप मुझसे क्या पूछना चाहते हैं? मैं अपनी पूरी क्षमता के साथ उत्तर देने की कोशिश करूँगा।  😊\n'

In [122]:
with_message_history =RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [123]:
config = {"configurable":{"session_id": "chat4"}}
response = with_message_history.invoke(
    {"messages":[HumanMessage(content="Hi, I am Aijul")],"language":"Hindi"},
    config=config
)
response.content

'नमस्ते अजय, मुझे खुशी है कि आप मुझसे बात कर रहे हैं! \n\nआप मुझसे जो भी प्रश्न पूछेंगे, मैं अपनी पूरी क्षमता के साथ उत्तर देने की कोशिश करूँगा।  \n\nआप क्या जानना चाहते हैं? 😊\n\n'

In [124]:
response = with_message_history.invoke(
    {"messages":[HumanMessage(content="what's my name?")],"language":"Hindi"},
    config=config
)
response.content

'आपका नाम अजय है, जैसा कि आपने खुद ही बताया है। 😊  \n'

In [126]:
!pip install transformers

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.5 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.5 MB 837.5 kB/s eta 0:00:12
   -- ------------------------------------- 0.5/10.5 MB 837.5 kB/s eta 0:00:12
   --- ------------------------------------ 0.8/10.5 MB 817.9 kB/s eta 0:00:12
   ---- ----------------------------------- 1.0/10.5 MB 853.0 kB/s eta 0:00:12
   ---- ----------------------------------- 1.0/10.5 MB 853.0 kB/s eta 0:00:12
   ----- ---------------------------------- 1.3/10.5 MB 849.7 kB/s eta 0:00:11
   ------ --------------------------------- 1.6/10.5 MB 847.3 kB/s eta 0:00:11
   ------- -------------------------------- 1.8/10.5 MB 853.0 kB/s eta 0:00:11
   ------- -------------------------------- 1.8/10.5 MB 853.0 kB/s eta 0:00:11
   -------- ------

### manage the conversation History

In [129]:
from langchain_core.messages import SystemMessage, trim_messages, HumanMessage, AIMessage
trimmer=trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="You are a good assistant"),
    HumanMessage(content="Hi, I am Aijul"),
    AIMessage(content="Hi"),
    HumanMessage(content="Hi, I like vanilla ice cream"),
    AIMessage(content="Nice"),
    HumanMessage(content="What is 2+2?"),
    AIMessage(content="4"),
    HumanMessage(content="Thanks"),
    AIMessage(content="No problem"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]


trimmer.invoke(messages)

[SystemMessage(content='You are a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi, I am Aijul', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi, I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is 2+2?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No problem', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [130]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
    "messages":messages + [HumanMessage(content="what ice cream do I like")],
    "language": "English"
    }
)

response.content

"You said you like vanilla ice cream! 😊  What's your favorite way to eat it?\n"

In [131]:
## wrap this messages history
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config ={"configurable": {"session_id": "chat5"}}


In [132]:
response = with_message_history.invoke(
    {"messages":[HumanMessage(content="what's my name?")],"language":"English"},
    config=config
)
response.content

"As an AI, I have no memory of past conversations and don't know your name. Could you tell me? 😊  \n\n"